In [140]:
from unstructured.partition.auto import partition
elements = partition("book-stat_first_5_pages.pdf")


print("\n\n".join([str(el) for el in elements]))

text = "\n\n".join([str(el) for el in elements])

GROQ_API = "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"

import os

from groq import Groq

client = Groq(
    api_key=GROQ_API,
)

model = "llama3-70b-8192"

requirements = """

Nếu bạn gặp heading khởi đầu như "Chương" hay "Phần", hoặc là heading có numbering, khi tóm tắt bạn hãy tạo ra các dấu hiệu để ngăn cách giữa các phần khác nhau.
Cấu trúc của dấu hiệu là "###" + "Tên heading" + "###" (không bao gồm dấu ngoặc kép). Ví dụ: "###Chương 1###" hoặc "###1.1###".
Khi bạn đọc và gặp dấu hiệu này, hãy hiểu rằng đó là một phần mới trong văn bản. Các dấu hiệu này chỉ để giúp bạn hiểu cấu trúc của văn bản, không phải là phần của văn bản gốc.
Chỉ trả về đoạn văn bản đã được xử lí, không được nói gì khác.
Output của bạn không được quá 2000 từ.
Khi tóm tắt, hãy bỏ qua các mục như chú thích, bảng biểu, hình ảnh, ví dụ, bài tập, các câu hỏi, câu đố hoặc bất kỳ phần nào không phải là văn bản chính.

"""


prompt_adding_summary_template = """
Bạn là một trợ lý ảo giúp người mù đọc sách. Nhiệm vụ của bạn là kết hợp hai đoạn văn bản: một đoạn văn bản mới chưa được tóm tắt và một đoạn văn bản đã được tóm tắt trước đó. Đoạn kết hợp cuối cùng phải bao gồm đầy đủ các thông tin quan trọng từ cả hai đoạn bằng cách sử dụng bất kỳ phương pháp nào, như tóm tắt, tổng hợp hoặc sắp xếp lại, miễn là đảm bảo giữ nguyên ý nghĩa và nội dung cốt lõi.

Bạn hãy trả về bản tóm tắt cuối cùng dưới dạng văn bản. Bạn không nói những thứ không liên quan. Bạn cũng không cần phải tạo ra một tóm tắt hoàn chỉnh, nhưng hãy chắc chắn rằng bản tóm tắt của bạn chứa đầy đủ thông tin quan trọng từ cả hai đoạn văn bản.

Bạn đừng nói lặp ý, cũng đừng tóm tắt quá dài. Hãy tập trung vào việc truyền đạt. Bạn không nói những thứ không có trong văn bản.

Nếu văn bản quá dài, bạn có thể không cần đi vào chi tiết.

{requirements}

Dưới đây là đoạn văn bản đã được tóm tắt trước đó:
{summary}

Dưới đây là đoạn văn bản mới chưa được tóm tắt:
{text}
"""

prompt_combine_summaries_template = """ Bạn là một trợ lý ảo giúp người mù đọc sách. Nhiệm vụ của bạn là kết hợp hai đoạn văn bản đã được tóm tắt: một đoạn văn bản tóm tắt trước đó và một đoạn văn bản tóm tắt mới. Đoạn kết hợp cuối cùng phải bao gồm đầy đủ các thông tin quan trọng từ cả hai đoạn tóm tắt bằng cách sử dụng bất kỳ phương pháp nào, như tổng hợp hoặc sắp xếp lại, miễn là đảm bảo giữ nguyên ý nghĩa và nội dung cốt lõi.

Bạn hãy trả về bản tóm tắt cuối cùng dưới dạng văn bản. Bạn không nói những thứ không liên quan. Bạn cũng không cần phải tạo ra một bản tóm tắt quá dài, nhưng hãy chắc chắn rằng bản tóm tắt của bạn chứa đầy đủ thông tin quan trọng từ cả hai đoạn tóm tắt.

Bạn đừng nói lặp ý, cũng đừng làm bản kết hợp quá chi tiết hoặc dư thừa. Hãy tập trung vào việc truyền đạt thông tin cốt lõi.

{requirements}

Dưới đây là đoạn văn bản đã được tóm tắt trước đó:
{summary}

Dưới đây là đoạn văn bản tóm tắt mới:
{new_summary}
"""

prompt_sumary = """
Bạn là một trợ lý ảo giúp người mù đọc sách. Nhiệm vụ của bạn là tóm tắt một đoạn văn bản.

Bạn hãy trả về bản tóm tắt cuối cùng dưới dạng văn bản. Bạn không nói những thứ không liên quan. Bạn cũng không cần phải tạo ra một bản tóm tắt quá dài, nhưng hãy chắc chắn rằng bản tóm tắt của bạn chứa đầy đủ thông tin quan trọng từ cả hai đoạn tóm tắt.

Bạn đừng nói lặp ý, cũng đừng làm bản kết hợp quá chi tiết hoặc dư thừa. Hãy tập trung vào việc truyền đạt thông tin cốt lõi.

{requirements}

Dưới đây là đoạn văn bản cần được tóm tắt:
{text}
"""

prompt_normalize = """
Bạn là một trợ lý ảo giúp người mù đọc sách. Nhiệm vụ của bạn là chuẩn hóa một đoạn văn bản.

Bạn hãy trả về bản văn bản cuối cùng dưới dạng văn bản. Bạn không nói những thứ không liên quan. Bạn cũng không cần phải tạo ra một bản văn bản quá dài, nhưng hãy chắc chắn rằng bản văn bản của bạn chứa đầy đủ thông tin cần thiết.

Đây là tính chất của văn bản cần chuẩn hóa:
{requirements}

Bạn hãy chuyển nó về dạng bình thường để người bình thường hiểu, không cần phải tóm tắt hoặc thêm thông tin. Bỏ đi các phần dấu hiệu, chú thích, hoặc thông tin không cần thiết.
Hãy loại bỏ những dấu hiệu của heading bắt đầu bằng "###" và kết thúc bằng "###". Ví dụ: "###Chương 1###" hoặc "###1.1###".
Đừng thêm xuống dòng không cần thiết. Loại bỏ các heading và viết thành đoạn văn bình thường.

Dưới đây là đoạn văn bản cần được chuẩn hóa:
{text}

"""

1. Giải tích tổ hợp

for ik in range (nk):

m += 1

Lời giải. Giá trị khởi tạo của m bằng 0. Khối lệnh này gồm k vòng lặp khác nhau. Sau mỗi bước lặp của từng vòng lặp giá trị của k được tăng lên một đơn vị. Gọi Ti là việc thi hành vòng lặp thứ i. Có thể làm Ti bằng ni cách vì vòng lặp thứ i có ni bước lặp. Do các vòng lặp không thể thực hiện đồng thời nên theo quy tắc cộng, giá trị cuối cùng của m bằng số cách thực hiện (cid:4) một trong số các nhiệm vụ Ti, tức là m = n1 + n2 + ... + nk.

Ngôn ngữ tập hợp Nếu A1, A2, ..., Ak là các tập hợp đôi một rời nhau, khi đó số phần tử của hợp các tập hợp này bằng tổng số các phần tử của các tập thành phần. Giả sử Ti là việc chọn một phần tử từ tập Ai với i = 1, 2, ..., k. Có |Ai| cách làm và không có hai việc nào có thể được làm cùng một lúc. Số cách chọn một phần tử của hợp các tập hợp này, một mặt bằng số phần tử của nó, mặt khác theo quy tắc cộng nó bằng |A1| + |A2| + ... + |Ak|. Do đó ta có: |A1 ∪ A2 ∪ ... ∪ Ak| = |A1| + |A2| + ... + |Ak|.


In [141]:
import time

def add_summarize(text, summary):
    sleep_timer = 2
    while True:
        try:
            completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt_adding_summary_template.format(text=text, summary=summary, requirements=requirements),
                    }
                ],
                model=model,
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(e)
            print("Sleeping for", sleep_timer, "seconds")
            time.sleep(sleep_timer)
            sleep_timer *= 2

def combine_summaries(summary, new_summary):
    sleep_timer = 2
    while True:
        try:
            completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt_combine_summaries_template.format(summary=summary, new_summary=new_summary, requirements=requirements),
                    }
                ],
                model=model,
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(e)
            print("Sleeping for", sleep_timer, "seconds")
            time.sleep(sleep_timer)
            sleep_timer *= 2

def summarize(text):
    sleep_timer = 2
    while True:
        try:
            completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt_sumary.format(text=text, requirements=requirements),
                    }
                ],
                model=model,
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(e)
            print("Sleeping for", sleep_timer, "seconds")
            time.sleep(sleep_timer)
            sleep_timer *= 2

def normalize(text):
    sleep_timer = 2
    while True:
        try:
            completion = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt_normalize.format(text=text, requirements=requirements),
                    }
                ],
                model=model,
            )
            return completion.choices[0].message.content
        except Exception as e:
            print(e)
            print("Sleeping for", sleep_timer, "seconds")
            time.sleep(sleep_timer)
            sleep_timer *= 2

In [142]:
summary = "Chưa có gì được tóm tắt"

# 6 elements
limit = 2000

# Combine the elements but not too much
temp = ""
new_elements = []
for el in elements:
    if len(str(el)) <= 2:
        continue
    if len(temp) + len(str(el)) > limit:
        new_elements.append(temp)
        temp = ""
    temp += str(el) + "\n\n"

new_elements.append(temp)

elements = new_elements

# print(elements)

# for el in elements:
#     if len(str(el)) <= 2:
#         continue
    
#     summary = summarize(str(el), summary)
#     print("-------------Done with element: ", el)
#     print("-------------Summary: ", summary)
    
# print(summary)

In [143]:
class BruteForceSummarizer:
    def __init__(self, elements):
        self.elements = elements
        self.summaries = []
    
    def process(self):
        for i in range(len(self.elements)):
            sumary = summarize(self.elements[i])
            self.summaries.append(sumary)
            print("-------------Done with element: ", i)
            
    def query(self, l, r, text=None):
        print("-------------Query: ", end="")
        for i in range(l, r+1):
            print(self.elements[i], end=", ")

        
        summarize = self.summaries[l]
        for i in range(l + 1, r+1):
            summarize = combine_summaries(summarize, self.summaries[i])
            print("-------------Done with element: ", i)
            
        if text is not None:
            summarize = add_summarize(text, summarize)
            print("-------------Done with element: ", "text")
        return summarize
    
# BS = BruteForceSummarizer(elements)
# BS.process()
# print(BS.query(0, 2, text='Bạn Hoàng là một người mù rất thích đọc sách.'))

import math

class SparseTableSummarizer:
    def __init__(self, elements, name):
        self.elements = elements
        self.name = name
        self.summaries = []
        self.n = len(elements)
        self.k = 10
        self.st = [["" for i in range(self.k)] for j in range(self.n)]
    
    
        
    def local_save(self):
        # utf-8
        with open(f"{self.name}.txt", "w", encoding="utf-8") as f:
            f.write(str(self.elements))
            f.write("\n")
            f.write(str(self.summaries))
            f.write("\n")
            f.write(str(self.n))
            f.write("\n")
            f.write(str(self.k))
            f.write("\n")
            f.write(str(self.st))
            f.write("\n")
            
    def local_load(self):
        with open(f"{self.name}.txt", "r", encoding="utf-8") as f:
            self.elements = eval(f.readline())
            self.summaries = eval(f.readline())
            self.n = int(f.readline())
            self.k = int(f.readline())
            self.st = eval(f.readline())
    
    def process(self):
        for i in range(self.n):
            summary = summarize(self.elements[i])
            self.summaries.append(summary)
            print("-------------Done adding summary for element: ", i)
            self.st[i][0] = summary
            
        for j in range(1, self.k):
            for i in range(self.n):
                if i + (1 << j) <= self.n:
                    self.st[i][j] = combine_summaries(self.st[i][j-1], self.st[i + (1 << (j-1))][j-1])
                    print(f"-------------Done combining summary for element: {i} and {j}")
                    
    def query(self, l, r, pretext = None, postText = None):
        print("-------------Query: ", end="")
        for i in range(l, r+1):
            print(self.elements[i], end=", ")
        
        len = r - l + 1
        k = 0
        while (1 << (k+1)) <= len:
            k += 1
            
        summarize = self.st[l][k]
        
        summarize = combine_summaries(summarize, self.st[r - (1 << k) + 1][k])
        
        if pretext is not None:
            summarize = add_summarize(pretext, summarize)
            print("-------------Done with element: ", "text")
            
        if postText is not None:
            summarize = add_summarize(summarize, postText)
            print("-------------Done with element: ", "text")
        return summarize

In [144]:
ST = SparseTableSummarizer(elements, "book-stat_first_5_pages")
ST.process()
ST.local_save()

-------------Done adding summary for element:  0
-------------Done adding summary for element:  1
-------------Done adding summary for element:  2
-------------Done adding summary for element:  3
-------------Done adding summary for element:  4
-------------Done adding summary for element:  5
-------------Done adding summary for element:  6
-------------Done adding summary for element:  7
-------------Done adding summary for element:  8
-------------Done adding summary for element:  9
-------------Done adding summary for element:  10
-------------Done adding summary for element:  11
-------------Done combining summary for element: 0 and 1
-------------Done combining summary for element: 1 and 1
-------------Done combining summary for element: 2 and 1
-------------Done combining summary for element: 3 and 1
-------------Done combining summary for element: 4 and 1
-------------Done combining summary for element: 5 and 1
-------------Done combining summary for element: 6 and 1
-----------

In [145]:
# Display the tree
for i in range(ST.n):
    print(f"Element {i}")
    print(ST.st[i][0])


result = ST.query(0, 5)
print("-------------------------------------------Now displaying the result")
print(normalize(result))

Element 0
###Giải tích tổ hợp###

Giá trị khởi tạo của m bằng 0 và sau mỗi bước lặp của từng vòng lặp, giá trị của k được tăng lên một đơn vị. Số cách thực hiện một trong số các nhiệm vụ Ti bằng số phần tử của hợp các tập hợp A1, A2, ..., Ak.

###Ngôn ngữ tập hợp###

Nếu A1, A2, ..., Ak là các tập hợp đôi một rời nhau, số phần tử của hợp các tập hợp này bằng tổng số các phần tử của các tập thành phần. Số cách chọn một phần tử của hợp các tập hợp này bằng |A1| + |A2| + ... + |Ak|.

###Quy tắc nhân###

Có n1.n2...nk cách thi hành nhiệm vụ đã cho, nếu việc Ti có thể làm bằng ni cách sau khi các việc T1, T2, ...Ti−1 đã được làm.

###Ví dụ 1.3###

Có 26.100 = 2600 cách khác nhau để gán nhãn cho một chiếc ghế.

###Ví dụ 1.4###

Có tổng cộng 2n chuỗi nhị phân khác nhau có độ dài bằng n, vì mỗi bit trong n bit của chuỗi nhị phân có thể chọn bằng hai cách.
Element 1
### Ví dụ 1.5. ###
Có thể tạo được nm ánh xạ từ tập A có m phần tử vào tập B có n phần tử.

### Ví dụ 1.6. ###
Có n! / (n - m)! đơ